# Details re: computing the mean-field dynamics

$$M\Theta^{t+1} \sim \textrm{Multinomial}\left(\boldsymbol{\alpha}(\Theta^t), M\right)$$

$$\alpha_r = 
\int_{x}\mathcal{N}\left(x; \mu_r, \sigma_r\right) \prod_{r'\neq r} \Phi\left(x; \mu_{r'}, \sigma_{r'}\right)$$

$$\mu_r = R \sum_{r'} \mu_{rr'} \Theta_{r'}^t + u_r^t$$

$$\sigma^2_r = R \sum_{r'} \gamma_{rr'}^2 \Theta_{r'}^t + v_r^t$$

We have to specify a finite integration domain, which we want to make sure reasonably covers the nonzero section of the integrand.

Each term in the integrand has a well-defined mean and variance.

The edge cases will occur when some $\Theta_r = 1$. Then (for no input) we have

$$|\mu_r|^{max} = R \times \text{max}\left(\mu_{rr'}^{max}, -\mu_{rr'}^{min}\right)$$

$$(\sigma_r^2)^{max} = R \times (\gamma_{rr'}^2)^{max}$$

To keep bounds at least $a$ stdevs beyond the widest Gaussian, we use

$$x_{max} = |\mu_r|^{max} + a\sqrt{(\sigma_r^2)^{max}}$$

$$x_{min} = -x_{max}$$

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import sys

from disp import set_plot

In [4]:
def get_x_bds(mus, gams, nsgm=3):
    r = mus.shape[0]
    mu_max = r*np.max([mus.max(), -mus.min()])
    gam_max = r*gams.max()
    
    x_max = mu_max + nsgm*gam_max
    
    return -x_max, x_max

In [9]:
mus = 2*np.eye(16)
gams = np.ones((16, 16))
gams[0, 0] = 10
get_x_bds(mus, gams)

(-512.0, 512.0)

In [2]:
# numeric params for computing MF quantities
DX = .001
X_MIN = -100
X_MAX = 100

X = np.linspace(X_MIN, X_MAX, int((X_MAX-X_MIN)/DX)+1)

def norm(x, mu, sgm):
    return stats.norm.pdf(x, loc=mu, scale=sgm)

def phi(x, mu, sgm):
    return stats.norm.cdf(x, loc=mu, scale=sgm)

def alph(th, u, v, R, mus, gams):
    """
    th \in [0, 1]^R
    u \in R^R
    v \in R_+^r
    R scalar
    mus \in R^{RxR}
    gams \in R^{RxR}
    """
    mu_r = R*(mus @ th) + u
    sgm_r = np.sqrt(R*((gams**2)@th) + v)
    
    phis = np.array([phi(X, mu_r_, sgm_r_) for mu_r_, sgm_r_ in zip(mu_r, sgm_r)])
    
    mnot_r = ~np.eye(len(th), dtype=bool) # masks for selecting all but one r
    
    th_next = np.nan*np.zeros(R)
    for r, (mu_r_, sgm_r_) in enumerate(zip(mu_r, sgm_r)):
        
        mask_r = mnot_r[r]
        th_next[r] = np.sum(norm(X, mu_r_, sgm_r_) * np.prod(phis[mask_r, :], axis=0))*DX
        
    return th_next